# Notebook to create our index

start elasticsearch with .\bin\elasticsearch.bat  (you have to be in the right folder)
use the browser to check if its activated (http://127.0.0.1:9200/)

In [1]:
# Import elasticsearch packages

from elasticsearch import Elasticsearch

# Elastic search configuation

es = Elasticsearch(HOST="http://localhost", PORT=9200)
es = Elasticsearch()

In [2]:
# define the mapping for the new index

my_mapping = {
  "mappings": {
    "properties": {
      "document_text": {
        "type": "text",     
        "similarity": "BM25" # BM25 is the default algorithm
      },
      "ocr_text" : {
        "type":"text"
      },

      #"cluster_when_5_clusters_in_100dim":{
      #  "type":"text",
      #  "similarity":"boolean"},

      #"cluster_when_7_clusters_in_100dim":{
      #  "type":"text",
      #  "similarity":"boolean"},

      #"cluster_when_10_clusters_in_100dim":{
      #  "type":"text",
      #  "similarity":"boolean"},

      #"cluster_when_12_clusters_in_100dim":{
      #  "type":"text",
      #  "similarity":"boolean"},

      #"cluster_when_20_clusters_in_100dim":{
      #  "type":"text",
      #  "similarity":"boolean"},

      #"cluster_when_30_clusters_in_100dim":{
      #  "type":"text",
      #  "similarity":"boolean"},

      #"cluster_when_5_clusters_in_2dim":{
      #  "type":"text",
      #  "similarity":"boolean"},

      #"cluster_when_7_clusters_in_2dim":{
      #  "type":"text",
      #  "similarity":"boolean"},

      #"cluster_when_10_clusters_in_2dim":{
      #  "type":"text",
      #  "similarity":"boolean"},

      #"cluster_when_12_clusters_in_2dim":{
      #  "type":"text",
      #  "similarity":"boolean"},

      #"cluster_when_20_clusters_in_2dim":{
      #  "type":"text",
      #  "similarity":"boolean"},

      #"cluster_when_30_clusters_in_2dim":{
      #  "type":"text",
      #  "similarity":"boolean"},

      "cluster_when_14_clusters_in_10dim":{
        "type":"text",
        "similarity":"boolean"},

      "sentiment":{   
         "type":"float"},

      "sentiment_html_title":{
        "type":"text",
        "similarity":"boolean"
      } 
    }
  }
}


In [34]:
# if it doesn't already exist, create the new index
# if it already exist delete it and create a new one (not necessary, but to better to be sure)
# normally index needs to be created once and then can be used again in the next session

if(es.indices.exists(index="final_boromir_index") == False):
   es.indices.create(index="final_boromir_index", body=my_mapping)
else:
    es.indices.delete(index="final_boromir_index")
    es.indices.create(index="final_boromir_index", body=my_mapping)

C:\Users\grego\AppData\Local\Programs\Python\Python39\lib\site-packages\elasticsearch\connection\base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
<ipython-input-34-16b0721943aa>:9: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index="final_boromir_index", body=my_mapping)


In [31]:
import pandas as pd

# get clusters, sentiments and ocr text for the ids -> these parts have been calculated in seperate notebooks and safed into tsv files
cluster_df = pd.read_csv("image_cluster14_10d.tsv", sep="\t")

sentiment_df = pd.read_csv("afinnSentimentNEW.tsv", sep="\t")

sentiment_title_df = pd.read_csv("htmlTitlesStanceNEW.tsv", sep="\t")

ocr_df = pd.read_csv("imge_texts_from_OCR_new.tsv", sep="\t")

In [24]:
import os
from os import listdir, mkdir, path
from PIL import Image

#cwd = os.getcwd()
#data_dir = cwd + "/data/images/"
#print(data_dir)

In [20]:

#str(ocr_df.loc[ocr_df['id'] == 'I2290ecf2e963023e'].values[0][1])


' percent hourly paid workers with earnings below the federal minimum wage age and sex annual averages mien cry total years years years years years years source bureau labor statistics'

In [32]:
# Fill nas from BERT modell
sentiment_title_df = sentiment_title_df.fillna('Neutral')

In [33]:
sentiment_title_df

,id,title,stance
0,I000330ba4ea0ad13,Comprehensive Review of Current and Upcoming A...,Positive
1,I000d8de9c4746ee9,File:DEA Raid protest.jpg - Wikimedia Commons,Negative
2,I0010d5b5473065fa,Vector Medical Poster Obesity. Reasons Of The ...,Positive
3,I001652ec040f07c4,Your Guide to LGBTQIAPK Addiction Treatment,Positive
4,I001a318b3176d501,i get paid below minimum wage after taxes : memes,Negative
...,...,...,...
23836,Ifff6a608e404c89b,Phelps doubts doping protests will bring real ...,Neutral
23837,Ifff77669db5d587b,Global 5G Protest Day - Life-Environmental Net...,Neutral
23838,Ifff91dfb5b0256e8,Fixing Canadas Unfixed Election Dates,Neutral
23839,Ifffa6ec57e1f11e1,The Euthanasia Coaster by recyclebin - Meme Ce...,Neutral


In [35]:
#directory of the task data
data_dir = "E:/InformationRetrieval/data_new/main/"
# find all preprocessed text files and add them to index
image_files = [f for f in listdir(data_dir)]
for f in image_files:
    subdir = data_dir + f
    subdir_files = [f for f in listdir(subdir)]
    print("\n", f)

    for f2 in subdir_files:
        #print("   ", subdir_files.index(f2), " von ", len(subdir_files))
        #print(f2)
        subsubdir = subdir + "/" + f2 + "/pages/"
        subsubdir_files = [f for f in listdir(subsubdir)]

        for f3 in subsubdir_files:
            subsubsubdir = subsubdir + "/" + f3 + "/snapshot/"
            
            # open text file
            with open(subsubsubdir + 'text_pp.txt', errors='ignore') as fp:
                doc = fp.readlines()
                doc = ''.join(e for e in doc)
                doc_as_map = {"document_text":doc,
                             "ocr_text":str(ocr_df.loc[ocr_df['id'] == f2].values[0][1]),

                            # 5 7 10 12 20 30
                             #"cluster_when_5_clusters_in_100dim":str(int(cluster_df.loc[cluster_df['id'] == f2].cluster_when_5_clusters_in_100dim)),
                             #"cluster_when_7_clusters_in_100dim":str(int(cluster_df.loc[cluster_df['id'] == f2].cluster_when_7_clusters_in_100dim)),
                             #"cluster_when_10_clusters_in_100dim":str(int(cluster_df.loc[cluster_df['id'] == f2].cluster_when_10_clusters_in_100dim)),
                             #"cluster_when_12_clusters_in_100dim":str(int(cluster_df.loc[cluster_df['id'] == f2].cluster_when_12_clusters_in_100dim)),
                             #"cluster_when_20_clusters_in_100dim":str(int(cluster_df.loc[cluster_df['id'] == f2].cluster_when_20_clusters_in_100dim)),
                             #"cluster_when_30_clusters_in_100dim":str(int(cluster_df.loc[cluster_df['id'] == f2].cluster_when_30_clusters_in_100dim)),

                             #"cluster_when_5_clusters_in_2dim":str(int(cluster_df.loc[cluster_df['id'] == f2].cluster_when_5_clusters_in_2dim)),
                             #"cluster_when_7_clusters_in_2dim":str(int(cluster_df.loc[cluster_df['id'] == f2].cluster_when_7_clusters_in_2dim)),
                             #"cluster_when_10_clusters_in_2dim":str(int(cluster_df.loc[cluster_df['id'] == f2].cluster_when_10_clusters_in_2dim)),
                             #"cluster_when_12_clusters_in_2dim":str(int(cluster_df.loc[cluster_df['id'] == f2].cluster_when_12_clusters_in_2dim)),
                             #"cluster_when_20_clusters_in_2dim":str(int(cluster_df.loc[cluster_df['id'] == f2].cluster_when_20_clusters_in_2dim)),
                             #"cluster_when_30_clusters_in_2dim":str(int(cluster_df.loc[cluster_df['id'] == f2].cluster_when_30_clusters_in_2dim)),

                             "cluster_when_14_clusters_in_10dim":str(int(cluster_df.loc[cluster_df['id'] == f2].cluster)),


                             "sentiment":float(sentiment_df.loc[sentiment_df['id'] == f2].AFINN),
                             
                             "sentiment_html_title":sentiment_title_df.loc[sentiment_title_df['id'] == f2].stance

                            }
                
                # pushes one document to the index
                es.index(index = "final_boromir_index", id = f2, document = doc_as_map)



 I00

 I01

 I02

 I03

 I04

 I05

 I06

 I07

 I08

 I09

 I0a

 I0b

 I0c

 I0d

 I0e

 I0f

 I10

 I11

 I12

 I13

 I14

 I15

 I16

 I17

 I18

 I19

 I1a

 I1b

 I1c

 I1d

 I1e

 I1f

 I20

 I21

 I22

 I23

 I24

 I25

 I26

 I27

 I28

 I29

 I2a

 I2b

 I2c

 I2d

 I2e

 I2f

 I30

 I31

 I32

 I33

 I34

 I35

 I36

 I37

 I38

 I39

 I3a

 I3b

 I3c

 I3d

 I3e

 I3f

 I40

 I41

 I42

 I43

 I44

 I45

 I46

 I47

 I48

 I49

 I4a

 I4b

 I4c

 I4d

 I4e

 I4f

 I50

 I51

 I52

 I53

 I54

 I55

 I56

 I57

 I58

 I59

 I5a

 I5b

 I5c

 I5d

 I5e

 I5f

 I60

 I61

 I62

 I63

 I64

 I65

 I66

 I67

 I68

 I69

 I6a

 I6b

 I6c

 I6d

 I6e

 I6f

 I70

 I71

 I72

 I73

 I74

 I75

 I76

 I77

 I78

 I79

 I7a

 I7b

 I7c

 I7d

 I7e

 I7f

 I80

 I81

 I82

 I83

 I84

 I85

 I86

 I87

 I88

 I89

 I8a

 I8b

 I8c

 I8d

 I8e

 I8f

 I90

 I91

 I92

 I93

 I94

 I95

 I96

 I97

 I98

 I99

 I9a

 I9b

 I9c

 I9d

 I9e

 I9f

 Ia0

 Ia1

 Ia2

 Ia3

 Ia4

 Ia5

 Ia

In [27]:
f2

'If907c89a769fa037'